In [165]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [166]:
import sys
import os
import torch
from functools import partial
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm

import torch.nn as nn
from embedding import CustomEmbedding
from transformers import EncoderDecoderTransformer
from utils import padding_collate_fn

from generate_data import RandomIntegerDataset

In [167]:
n_real_tokens = 10
PAD_TOKEN_IDX = n_real_tokens
SOS_TOKEN_IDX = n_real_tokens + 1
EOS_TOKEN_IDX = n_real_tokens + 2
vocab_size = n_real_tokens + 3
D_MODEL = 16

embeddings = CustomEmbedding(vocab_size = vocab_size + 3, d_model = D_MODEL) # 3 = PAD, SOS, EOS

indices = torch.tensor([1,9])

# print(embeddings.embeddings.weight)
print(embeddings.embeddings(indices))

tensor([[ 0.8447, -1.1799, -1.1498,  0.6752,  0.2597, -1.1586, -0.0197, -0.6132,
          0.6359,  0.5050,  1.8588,  2.4235,  0.7090, -0.7020,  0.7179, -0.5701],
        [-0.6702,  0.9876, -0.2088, -2.3131, -0.3731,  0.8684,  1.6219, -0.0676,
         -1.3708, -0.8335,  0.7961,  0.6606, -0.3125,  0.6729, -0.5466,  0.1270]],
       grad_fn=<EmbeddingBackward0>)


In [168]:
MAX_CONTEXT_WINDOW = 50

BATCH_SIZE = 32
MIN_SEQ_LEN = 2
MAX_SEQ_LEN = min(10, MAX_CONTEXT_WINDOW)

NUM_TRAINING_SEQUENCES = 10000
NUM_VALIDATION_SEQUENCES = 1000

VOCAB = [i for i in range(10)] # non-sos,eos,pad

train_rand_ds = RandomIntegerDataset(MIN_SEQ_LEN, MAX_SEQ_LEN, NUM_TRAINING_SEQUENCES, VOCAB)
train_dataloader = DataLoader(train_rand_ds, batch_size = BATCH_SIZE, shuffle = True, collate_fn = partial(padding_collate_fn, pad_token_idx = PAD_TOKEN_IDX))

val_rand_ds = RandomIntegerDataset(MIN_SEQ_LEN, MAX_SEQ_LEN, NUM_VALIDATION_SEQUENCES, VOCAB)
val_dataloader = DataLoader(val_rand_ds, batch_size = BATCH_SIZE, shuffle = True, collate_fn = partial(padding_collate_fn, pad_token_idx = PAD_TOKEN_IDX))

In [169]:
input, label = next(iter(train_dataloader))
print(input[0])
print(input[1])
print(label)

tensor([[ 3,  7,  4,  7,  5,  6, 10, 10, 10, 10],
        [ 6,  7,  9,  0,  5, 10, 10, 10, 10, 10],
        [ 3,  6,  6,  2, 10, 10, 10, 10, 10, 10],
        [ 2,  5,  7,  8,  1,  5, 10, 10, 10, 10],
        [ 7,  1,  8, 10, 10, 10, 10, 10, 10, 10],
        [ 1,  7,  6,  6,  3,  8,  0,  1,  6,  9],
        [ 9,  9, 10, 10, 10, 10, 10, 10, 10, 10],
        [ 2,  7,  3,  9,  9,  3,  3,  7, 10, 10],
        [ 5,  5,  0,  3,  4,  2,  7,  3, 10, 10],
        [ 9,  2,  3,  5,  6,  0,  8,  7,  7,  2],
        [ 6,  1,  4,  8,  2,  8,  9,  3,  2,  2],
        [ 2,  3,  7, 10, 10, 10, 10, 10, 10, 10],
        [ 6,  5,  5,  7,  9,  3,  5,  5,  0, 10],
        [ 3,  7,  1,  0,  6,  6, 10, 10, 10, 10],
        [ 1,  4,  5,  7,  6,  3, 10, 10, 10, 10],
        [ 9,  0,  2,  2,  2,  1,  8,  6,  3, 10],
        [ 1,  8,  3,  9,  8, 10, 10, 10, 10, 10],
        [ 2,  1,  2,  0,  3, 10, 10, 10, 10, 10],
        [ 5,  6,  0,  1, 10, 10, 10, 10, 10, 10],
        [ 5,  4, 10, 10, 10, 10, 10, 10, 10, 10],


In [170]:
loss_fn = nn.CrossEntropyLoss(ignore_index = PAD_TOKEN_IDX, reduction = 'sum')

model = EncoderDecoderTransformer(
                    embeddings = embeddings, 
                    vocab_size = vocab_size, 
                    d_model = D_MODEL, 
                    num_attention_heads = 4, 
                    num_encoder_layers = 1, 
                    num_decoder_layers = 1, 
                    dim_feedforward = 32, 
                    max_context_window = MAX_CONTEXT_WINDOW)

optim = torch.optim.SGD(params = model.parameters(), lr = 1e-3, momentum = 0.9, weight_decay = 1e-4)

In [171]:
def run_train_epoch(dataloader: DataLoader, model: nn.Module, loss_fn: nn.Module, optimizer: torch.optim.Optimizer, calculate_sequence_accuracy: bool = False, calculate_token_accuracy: bool = False):
    """
    """
    model.train()

    num_sequences = len(dataloader.dataset)
    num_tokens = 0

    epoch_loss = 0.0
    total_correct_sequences = 0
    total_correct_tokens = 0

    for (source, target), label in tqdm(dataloader):

        # FORWARD
        pred_logits = model(source, target)

        # pred_logits.shape: [batch_size, seq_len, vocab_size]
        # label.shape: [batch_size, seq_len]

        # CrossEntropyLoss (loss_fn) only takes 2D predictions (n_batch * seq_len, vocab_size) and 1D labels (n_batch * seq_len)
        batch_loss = loss_fn(pred_logits.view(-1, pred_logits.size(-1)), label.view(-1))

        # LOG
        with torch.no_grad():
            epoch_loss += batch_loss.item()

            predictions = torch.argmax(pred_logits, dim = -1) # predictions.shape: [batch_size, seq_len]
            match_matrix = torch.eq(predictions, label)

            if calculate_sequence_accuracy:
                num_correct_sequences = torch.all(match_matrix, dim = 1).sum()
                total_correct_sequences += num_correct_sequences.item()

            if calculate_token_accuracy:
                num_correct_tokens = match_matrix.sum()      
                total_correct_tokens += num_correct_tokens.item()

                num_tokens += torch.numel(label)

        # BACKWARD
        batch_loss.backward()

        # OPTIMIZE
        optimizer.step()
        optimizer.zero_grad()

    average_epoch_loss = epoch_loss / num_sequences
    average_epoch_sequence_accuracy = total_correct_sequences / num_sequences if calculate_sequence_accuracy else None
    average_epoch_token_accuracy = total_correct_tokens / num_tokens if calculate_token_accuracy else None

    return average_epoch_loss, average_epoch_sequence_accuracy, average_epoch_token_accuracy

def run_gold_validation_loop(dataloader: DataLoader, model: nn.Module, loss_fn: nn.Module, calculate_sequence_accuracy: bool = False, calculate_token_accuracy: bool = False):
    """
    """
    model.eval()

    num_sequences = len(dataloader.dataset)
    num_tokens = 0

    epoch_loss = 0.0
    total_correct_sequences = 0
    total_correct_tokens = 0

    with torch.no_grad():
        
        for (source, target), label in tqdm(dataloader):
            
            # FORWARD
            pred_logits = model(source, target)
            batch_loss = loss_fn(pred_logits.view(-1, pred_logits.size(-1)), label.view(-1))

            # LOG
            epoch_loss += batch_loss.item()

            predictions = torch.argmax(pred_logits, dim = -1) # predictions.shape: [batch_size, seq_len]
            match_matrix = torch.eq(predictions, label)

            if calculate_sequence_accuracy:
                num_correct_sequences = torch.all(match_matrix, dim = 1).sum()
                total_correct_sequences += num_correct_sequences.item()

            if calculate_token_accuracy:
                num_correct_tokens = match_matrix.sum()      
                total_correct_tokens += num_correct_tokens.item()

                num_tokens += torch.numel(label)

    average_epoch_loss = epoch_loss / num_sequences
    average_epoch_sequence_accuracy = total_correct_sequences / num_sequences if calculate_sequence_accuracy else None
    average_epoch_token_accuracy = total_correct_tokens / num_tokens if calculate_token_accuracy else None

    return average_epoch_loss, average_epoch_sequence_accuracy, average_epoch_token_accuracy

def run_autoregressive_validation_loop(dataloader: DataLoader, model: nn.Module, loss_fn: nn.Module):
    """
    """
    model.eval()

    with torch.no_grad():
        ...

In [172]:
EPOCHS = 10

training_losses = list()
training_sequence_accuracies = list()
training_token_accuracies = list()

gold_validation_losses = list()
gold_validation_sequence_accuracies = list()
gold_validation_token_accuracies = list()

for i in range(EPOCHS):
    # print(f'Running epoch {i+1}...')

    training_loss, training_sequence_accuracy, training_token_accuracy = run_train_epoch(train_dataloader, model, loss_fn, optim, calculate_sequence_accuracy = True, calculate_token_accuracy = True)

    training_losses.append(training_loss)
    training_sequence_accuracies.append(training_sequence_accuracy)
    training_token_accuracies.append(training_token_accuracy)

    gold_val_loss, gold_val_sequence_accuracy, gold_val_token_accuracy = run_gold_validation_loop(val_dataloader, model, loss_fn, calculate_sequence_accuracy = True, calculate_token_accuracy = True)
    gold_validation_losses.append(gold_val_loss)
    gold_validation_sequence_accuracies.append(gold_val_sequence_accuracy)
    gold_validation_token_accuracies.append(gold_val_token_accuracy)

print(training_losses)
print(training_sequence_accuracies)
print(training_token_accuracies)

print()

print(gold_validation_losses)
print(gold_validation_sequence_accuracies)
print(gold_validation_token_accuracies)

100%|██████████| 32/32 [00:00<00:00, 349.61it/s]

[4.4000208343505856, 2.8920938091278074, 2.423911962890625, 2.2449683528900146, 2.143990826797485, 2.0492821838378905, 2.1396545362472534, 1.694596474838257, 1.784644213294983, 1.6451517740249635]
[0.0029, 0.0089, 0.0101, 0.0124, 0.0174, 0.0184, 0.0201, 0.0261, 0.0225, 0.0302]
[0.48437272064186726, 0.5365107127240927, 0.5543331631908998, 0.559940241947238, 0.5655634573304158, 0.5690386857059595, 0.5667785724711721, 0.5806307456588355, 0.5770323426573427, 0.5824041684885585]

[2.0652684020996093, 3.4530548887252808, 1.7810504817962647, 2.9549019260406495, 3.033310651779175, 2.449925817489624, 2.502387716293335, 4.811099613189698, 1.3951188898086548, 0.7931976718902588]
[0.012, 0.003, 0.017, 0.007, 0.002, 0.006, 0.006, 0.002, 0.021, 0.066]
[0.5636826783114993, 0.535518626734843, 0.5817272727272728, 0.5288181818181819, 0.5402078774617067, 0.5476363636363636, 0.556495633187773, 0.5137262079062958, 0.5891818181818181, 0.6110503282275711]
